In [1]:
# Parameters

ATTEMPTS = [0, 1, 2, 3, 4]
SECONDS = 1
INPUT_SIZE = 4096 * SECONDS

In [2]:
# Load the names of different samples

train_sample_names = []
test_splits_w = []
test_splits_n = []

for attempt in ATTEMPTS:
    train_sample_names.append([])
    test_splits_w.append([])
    test_splits_n.append([])
    with open("data/attempt%d/split_info_%d_sec_%d.txt" % (attempt, SECONDS, attempt)) as f:
        for line in f.readlines():
            line = line.strip()
            line = line.split(": ")
            if line[0] == "gw_test_ids":
                test_splits_w[-1] += eval(line[1])
            elif line[0] == "noise_test_ids":
                test_splits_n[-1] += eval(line[1])
            elif line[0] == "noise_train_ids":
                train_sample_names[-1] += eval(line[1])

In [3]:
# Load waves
import pandas as pd

train_splits = []
test_splits = []

for attempt in ATTEMPTS:
    all_samples = pd.read_csv("data/attempt%d/test_%d_sec_%d.csv" % (attempt, SECONDS, attempt)).values.tolist()
    all_samples += pd.read_csv("data/attempt%d/train_%d_sec_%d_1.csv" % (attempt, SECONDS, attempt)).values.tolist()
    all_samples += pd.read_csv("data/attempt%d/train_%d_sec_%d_2.csv" % (attempt, SECONDS, attempt)).values.tolist()
    
    train_splits.append([s[3:] for s in all_samples if s[1] in train_sample_names[attempt]])
    
    test_splits.append([])
    test_splits[-1] += [(s[3:], (0, 1)) for s in all_samples if s[1] in test_splits_w[attempt]]
    test_splits[-1] += [(s[3:], (1, 0)) for s in all_samples if s[1] in test_splits_n[attempt]]
    
assert len(train_splits[0][0]) == 4096 * SECONDS and len(test_splits[0][0][0]) == 4096 * SECONDS

In [4]:
# Save to big pickle file
from pickle import dump

dump(([[s[0] for s in t] for t in test_splits], [[s[1] for s in t] for t in test_splits]), open("data/test_%d_%d.pickle" % (INPUT_SIZE, SECONDS), "wb+"))
dump(train_splits, open("data/train_%d_%d.pickle" % (INPUT_SIZE, SECONDS), "wb+"))